# **EA2: Practico**

## **Problema 1**

Pokemon ha sido un juego de video y cartas que ha estado en el radar de muchos jóvenes y adultos, al grado que actualmente se ha visto un auge en la versión competitiva en los juegos de video. Un pokemon cuenta con diferentes características que lo hacen único, entre ellas se encuentran el tipo de pokemon, el ataque, la defensa, la velocidad, la vida, entre otros. En la versión competitiva se busca tener un equipo balanceado que pueda enfrentar a cualquier tipo de pokemon, por lo que se busca tener un equipo que tenga un balance entre ataque, defensa, velocidad y vida. Otros elementos que se buscan en un equipo son el tipo de pokemon, ya que algunos tipos son más fuertes que otros, por lo que se busca tener un equipo que pueda enfrentar a cualquier tipo de pokemon, el cual tiene tipos principales y secundarios

### **Analisis del Problema**

Resumiendo el problema que nos es presentado, tenemos que encontrar un equipo de pokemon optimo el cual tenga un balance en sus estadisticas de ataque, defensa, velocidad y vida. Como no es mencionado en el problema, asumiremos que el ataque y la defensa es el promedio de las estadisticas de ataque fisico  y especial, al igual que defensa fisica y especial para poder usar la mayor cantidad de datos posibles de los pokemones, esto se convertira de vuelta a int ya que no hay estadisticas floats.
Con esto, tambien tendremos que formar un equipo balanceado en cuanto a tipos, esto para que cubra una mayor area de ataque y defensa a otros equipos.

### **Preguntas**

Las preguntas que se nos pueden plantear en la resolucion de este problema son, 

¿El equipo debe tener solo pokemones con altas estadisticas totales, o especificas?

¿Que equipo cubre la mayor cantidad de tipos contra los que es efectivo?

¿Cuantos de los pokemones seran legendarios, o importara en el momento de agrupar el equipo?

¿Sera mas importante tener pokemones de 1 solo tipo o de 2, o dependera de estadisticas?

¿Que generacion de pokemones tiene los mas fuertes?

**Cargar librerias**

In [92]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import kmapper as km
import math
import warnings
import sklearn

warnings.simplefilter("ignore")

In [93]:
df = pd.read_csv("../Datos/Pokemon.csv")

In [94]:
# Agregar promedio de attack y special attack, al igual don defense
df["avg_attack"] = (df["Attack"] + df["Sp. Atk"]) / 2
df["avg_defense"] = (df["Defense"] + df["Sp. Def"]) / 2
# Redondear los valores
df["avg_attack"] = df["avg_attack"].apply(lambda x: math.ceil(x))
df["avg_defense"] = df["avg_defense"].apply(lambda x: math.ceil(x))

### **Analisis Exploratorio**

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   #            800 non-null    int64 
 1   Name         800 non-null    object
 2   Type 1       800 non-null    object
 3   Type 2       414 non-null    object
 4   Total        800 non-null    int64 
 5   HP           800 non-null    int64 
 6   Attack       800 non-null    int64 
 7   Defense      800 non-null    int64 
 8   Sp. Atk      800 non-null    int64 
 9   Sp. Def      800 non-null    int64 
 10  Speed        800 non-null    int64 
 11  Generation   800 non-null    int64 
 12  Legendary    800 non-null    bool  
 13  avg_attack   800 non-null    int64 
 14  avg_defense  800 non-null    int64 
dtypes: bool(1), int64(11), object(3)
memory usage: 88.4+ KB


In [96]:
df.describe()

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,avg_attack,avg_defense
count,800.000000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000,800.000000,800.000000
mean,362.813750,435.10250,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375,76.100000,73.013750
std,208.343798,119.96304,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129,27.220786,25.661044
min,1.000000,180.00000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000,10.000000,18.000000
25%,184.750000,330.00000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000,56.000000,55.000000
50%,364.500000,450.00000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000,73.500000,70.000000
75%,539.250000,515.00000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000,91.000000,90.000000
max,721.000000,780.00000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000,180.000000,230.000000


In [97]:
# Agregamos columna booleana que nos dice si tiene segundo tipo
df["no_type_2"] = df["Type 2"].isnull()
# Arreglamos los valores nulos del dataframe
df["Type 2"] = df["Type 2"].fillna("None")

In [98]:
# Creamos un dataframe numerico
df_num = df.select_dtypes(include=[np.number])
df_num.drop(
    ["Attack", "Generation", "Defense", "Sp. Atk", "Sp. Def"],
    axis=1,
    inplace=True,
)
df_num.drop_duplicates(keep="first", inplace=True)
df_num.info()

<class 'pandas.core.frame.DataFrame'>
Index: 791 entries, 0 to 799
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   #            791 non-null    int64
 1   Total        791 non-null    int64
 2   HP           791 non-null    int64
 3   Speed        791 non-null    int64
 4   avg_attack   791 non-null    int64
 5   avg_defense  791 non-null    int64
dtypes: int64(6)
memory usage: 43.3 KB


In [99]:
# Creamos una grafica con el avg attack, avg defense, vida y velocidad a traves de los ids
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df["#"], y=df["avg_attack"], mode="markers", name="avg_attack")
)
fig.add_trace(
    go.Scatter(x=df["#"], y=df["avg_defense"], mode="markers", name="avg_defense")
)
fig.add_trace(go.Scatter(x=df["#"], y=df["HP"], mode="markers", name="HP"))
fig.add_trace(go.Scatter(x=df["#"], y=df["Speed"], mode="markers", name="Speed"))
fig.update_layout(
    title="Pokemon Stats", xaxis_title="Pokemon ID", yaxis_title="Stat Value"
)
fig.show()

In [100]:
df_num.corr()

,#,Total,HP,Speed,avg_attack,avg_defense
#,1.000000,0.112637,0.095349,0.003468,0.109019,0.096116
Total,0.112637,1.000000,0.620381,0.574476,0.887109,0.761651
HP,0.095349,0.620381,1.000000,0.174774,0.467450,0.357721
Speed,0.003468,0.574476,0.174774,1.000000,0.511031,0.146585
avg_attack,0.109019,0.887109,0.467450,0.511031,1.000000,0.489667
avg_defense,0.096116,0.761651,0.357721,0.146585,0.489667,1.000000


In [101]:
# Sacamos los maximos de las estadisticas que se piden y las graficamos
hp = df_num["HP"].idxmax()
atk = df_num["avg_attack"].idxmax()
dfnse = df_num["avg_defense"].idxmax()
speed = df_num["Speed"].idxmax()
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["#"], y=df["HP"], mode="markers", name="HP"))
fig.add_trace(
    go.Scatter(x=df["#"], y=df["avg_attack"], mode="markers", name="avg_attack")
)
fig.add_trace(
    go.Scatter(x=df["#"], y=df["avg_defense"], mode="markers", name="avg_defense")
)
fig.add_trace(go.Scatter(x=df["#"], y=df["Speed"], mode="markers", name="Speed"))
fig.update_layout(
    title="Pokemon Stats", xaxis_title="Pokemon ID", yaxis_title="Stat Value"
)
fig.add_trace(
    go.Scatter(
        x=[hp],
        y=[df["HP"][hp]],
        mode="markers",
        name="Max HP",
        marker=dict(color="red", size=10),
    )
)
fig.add_trace(
    go.Scatter(
        x=[atk],
        y=[df["avg_attack"][atk]],
        mode="markers",
        name="Max Attack",
        marker=dict(color="red", size=10),
    )
)
fig.add_trace(
    go.Scatter(
        x=[dfnse],
        y=[df["avg_defense"][dfnse]],
        mode="markers",
        name="Max Defense",
        marker=dict(color="red", size=10),
    )
)
fig.add_trace(
    go.Scatter(
        x=[speed],
        y=[df["Speed"][speed]],
        mode="markers",
        name="Max Speed",
        marker=dict(color="red", size=10),
    )
)
fig.show()

In [102]:
# Sacamos el nombre de los pokemones de los maximos desde un df nuevo con las columnas que no tienen el df numerico
df_car = df[["#", "Name", "Type 1", "Type 2", "Legendary", "no_type_2", "Generation"]]
df_car.loc[[hp, atk, dfnse, speed]]

,#,Name,Type 1,Type 2,Legendary,no_type_2,Generation
261,242,Blissey,Normal,None,False,True,2
426,384,RayquazaMega Rayquaza,Dragon,Flying,True,False,3
230,213,Shuckle,Bug,Rock,False,False,2
431,386,DeoxysSpeed Forme,Psychic,None,True,True,3


Con este primer analisis podemos ver que solo 2 de los pokemones con estadisticas mas altas son legendarios, y tambien solo tienen 1 tipo 2 de estos. Dividiendolo en mitad y mitad. Esto nos da un espectro para contestar las ultimas 2 preguntas planteadas, ya que vemos que un equipo no necesita pokemones legendarios necesariamente para que sea bueno. Tambien nos damos cuenta que muchos pokemones buenos van a tener solo un tipo, por lo que se necesitara hacer un analisis, o por preferencia de si mantener un pokemon con 2 tipos o con mayores estadisticas.

Tambien podemos contestarnos de una manera no tan informada la 3er pregunta, ya que en estos pokemones vemos que los de mayor estadisticas son de la generacion 2 y 3.

In [103]:
# Hacer un diccionario de los tipos de pokemones y poner los numeros en un dataframe nuevo con solo las columnas de interes
df2 = df[
    [
        "#",
        "Type 1",
        "Type 2",
        "Total",
        "HP",
        "avg_attack",
        "avg_defense",
        "Speed",
    ]
]
type_dict = {}
for i in df2["Type 1"].unique():
    type_dict[i] = len(type_dict)
for i in df2["Type 2"].unique():
    if i not in type_dict:
        type_dict[i] = len(type_dict)
for key in type_dict:
    type_dict[key] += 1
type_dict["None"] = 0
df2["Type 1"] = df2["Type 1"].apply(lambda x: type_dict[x])
df2["Type 2"] = df2["Type 2"].apply(lambda x: type_dict[x])
df2.head()

,#,Type 1,Type 2,Total,HP,avg_attack,avg_defense,Speed
0,1,1,6,318,45,57,57,45
1,2,1,6,405,60,71,72,60
2,3,1,6,525,80,91,92,80
3,3,1,6,625,80,111,122,80
4,4,2,0,309,39,56,47,65


### **Diccionario de tipos**

{'Grass': 1,
 'Fire': 2,
 'Water': 3,
 'Bug': 4,
 'Normal': 5,
 'Poison': 6,
 'Electric': 7,
 'Ground': 8,
 'Fairy': 9,
 'Fighting': 10,
 'Psychic': 11,
 'Rock': 12,
 'Ghost': 13,
 'Ice': 14,
 'Dragon': 15,
 'Dark': 16,
 'Steel': 17,
 'Flying': 18,
 'None': 0}

Para poder incluir la fuerza y debilidad de los tipos de los pokemones, agregaremos un diccionario con una proporcion de efectividad en ataque, dividiendo esto entre la efectividad maxima la cual seria sun tipo super efectivo contra todo, esto se hara tambien con la defensa.

Esto se hara con el proposito de sacar valores mas reales de ataque y defensa con respecto a su tipo, incluyendo ambos tipos, ya que las proporciones de ambos tipos se multiplicaran.

In [104]:
attack_sum = {
    "Grass": (7 * 0.5 + 3 * 2 + 8) / 36,
    "Fire": (4 * 0.5 + 4 * 2 + 10) / 36,
    "Water": (3 * 0.5 + 3 * 2 + 12) / 36,
    "Bug": (7 * 0.5 + 3 * 2 + 8) / 36,
    "Normal": (2 * 0.5 + 0 * 2 + 16) / 36,
    "Poison": (7 * 0.5 + 3 * 2 + 7) / 36,
    "Electric": (3 * 0.5 + 2 * 2 + 12) / 36,
    "Ground": (2 * 0.5 + 5 * 2 + 10) / 36,
    "Fairy": (3 * 0.5 + 3 * 2 + 12) / 36,
    "Fighting": (2 * 0.5 + 4 * 2 + 12) / 36,
    "Psychic": (2 * 0.5 + 2 * 2 + 13) / 36,
    "Rock": (3 * 0.5 + 4 * 2 + 11) / 36,
    "Ghost": (1 * 0.5 + 2 * 2 + 14) / 36,
    "Ice": (4 * 0.5 + 4 * 2 + 10) / 36,
    "Dragon": (1 * 0.5 + 1 * 2 + 15) / 36,
    "Dark": (3 * 0.5 + 2 * 2 + 13) / 36,
    "Steel": (4 * 0.5 + 3 * 2 + 11) / 36,
    "Flying": (3 * 0.5 + 3 * 2 + 12) / 36,
    "None": 0,
}

In [105]:
defense_sum = {
    "Grass": (5 * 0.5 + 4 * 2 + 9) / 36,
    "Fire": (3 * 0.5 + 5 * 2 + 13) / 36,
    "Water": (2 * 0.5 + 4 * 2 + 12) / 36,
    "Bug": (3 * 0.5 + 3 * 2 + 12) / 36,
    "Normal": (1 * 0.5 + 1 * 2 + 16 + 1) / 36,
    "Poison": (2 * 0.5 + 5 * 2 + 11) / 36,
    "Electric": (1 * 0.5 + 3 * 2 + 14) / 36,
    "Ground": (3 * 0.5 + 2 * 2 + 13 + 1) / 36,
    "Fairy": (2 * 0.5 + 3 * 2 + 13 + 1) / 36,
    "Fighting": (3 * 0.5 + 3 * 2 + 12) / 36,
    "Psychic": (3 * 0.5 + 2 * 2 + 13) / 36,
    "Rock": (5 * 0.5 + 4 * 2 + 9) / 36,
    "Ghost": (2 * 0.5 + 2 * 2 + 14 + 2) / 36,
    "Ice": (4 * 0.5 + 1 * 2 + 13) / 36,
    "Dragon": (3 * 0.5 + 4 * 2 + 11) / 36,
    "Dark": (2 * 0.5 + 3 * 2 + 13 + 1) / 36,
    "Steel": (3 * 0.5 + 10 * 2 + 5 + 1) / 36,
    "Flying": (3 * 0.5 + 3 * 2 + 12 + 1) / 36,
    "None": 0,
}

In [106]:
df2["Attack Type 1"] = df["Type 1"].apply(lambda x: attack_sum[x])
df2["Attack Type 2"] = df["Type 2"].apply(lambda x: attack_sum[x])
df2["Defense Type 1"] = df["Type 1"].apply(lambda x: defense_sum[x])
df2["Defense Type 2"] = df["Type 2"].apply(lambda x: defense_sum[x])
df2.head(10)

,#,Type 1,Type 2,Total,HP,avg_attack,avg_defense,Speed,Attack Type 1,Attack Type 2,Defense Type 1,Defense Type 2
0,1,1,6,318,45,57,57,45,0.486111,0.458333,0.541667,0.611111
1,2,1,6,405,60,71,72,60,0.486111,0.458333,0.541667,0.611111
2,3,1,6,525,80,91,92,80,0.486111,0.458333,0.541667,0.611111
3,3,1,6,625,80,111,122,80,0.486111,0.458333,0.541667,0.611111
4,4,2,0,309,39,56,47,65,0.555556,0.000000,0.680556,0.000000
5,5,2,0,405,58,72,62,80,0.555556,0.000000,0.680556,0.000000
6,6,2,18,534,78,97,82,100,0.555556,0.541667,0.680556,0.569444
7,6,2,15,634,78,130,98,100,0.555556,0.486111,0.680556,0.569444
8,6,2,18,634,78,132,97,100,0.555556,0.541667,0.680556,0.569444
9,7,3,0,314,44,49,65,43,0.541667,0.000000,0.583333,0.000000


Sumamos los 2 tipos que existen del pokemon, favoreciendo los que tienen 2 tipos ya que cubren mas efectividades. 

In [107]:
df2["Attack Type"] = df2["Attack Type 1"] + df2["Attack Type 2"]
df2["Defense Type"] = df2["Defense Type 1"] + df2["Defense Type 2"]
df2.drop(
    ["Attack Type 1", "Attack Type 2", "Defense Type 1", "Defense Type 2"],
    axis=1,
    inplace=True,
)
df2.head(10)

,#,Type 1,Type 2,Total,HP,avg_attack,avg_defense,Speed,Attack Type,Defense Type
0,1,1,6,318,45,57,57,45,0.944444,1.152778
1,2,1,6,405,60,71,72,60,0.944444,1.152778
2,3,1,6,525,80,91,92,80,0.944444,1.152778
3,3,1,6,625,80,111,122,80,0.944444,1.152778
4,4,2,0,309,39,56,47,65,0.555556,0.680556
5,5,2,0,405,58,72,62,80,0.555556,0.680556
6,6,2,18,534,78,97,82,100,1.097222,1.250000
7,6,2,15,634,78,130,98,100,1.041667,1.250000
8,6,2,18,634,78,132,97,100,1.097222,1.250000
9,7,3,0,314,44,49,65,43,0.541667,0.583333


In [108]:
df2["adj_attack"] = df2["avg_attack"] * df2["Attack Type"]
df2["adj_defense"] = df2["avg_defense"] * df2["Defense Type"]
df2["adj_attack"] = df2["adj_attack"].apply(lambda x: math.ceil(x))
df2["adj_defense"] = df2["adj_defense"].apply(lambda x: math.ceil(x))
df2["adj_total"] = df2["adj_attack"] + df2["adj_defense"] + df2["HP"] + df2["Speed"]

In [109]:
df2.drop(
    ["avg_attack", "avg_defense", "Attack Type", "Defense Type", "Total"],
    axis=1,
    inplace=True,
)

In [110]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   #            800 non-null    int64
 1   Type 1       800 non-null    int64
 2   Type 2       800 non-null    int64
 3   HP           800 non-null    int64
 4   Speed        800 non-null    int64
 5   adj_attack   800 non-null    int64
 6   adj_defense  800 non-null    int64
 7   adj_total    800 non-null    int64
dtypes: int64(8)
memory usage: 50.1 KB


In [111]:
df2.describe()

,#,Type 1,Type 2,HP,Speed,adj_attack,adj_defense,adj_total
count,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,362.813750,7.306250,5.918750,69.258750,68.277500,61.783750,65.350000,264.670000
std,208.343798,4.876955,6.952163,25.534669,29.060474,33.484489,36.001947,89.004865
min,1.000000,1.000000,0.000000,1.000000,5.000000,5.000000,11.000000,92.000000
25%,184.750000,3.000000,0.000000,50.000000,45.000000,36.000000,37.000000,195.000000
50%,364.500000,5.000000,1.000000,65.000000,65.000000,53.000000,55.000000,252.000000
75%,539.250000,11.000000,11.000000,80.000000,90.000000,83.250000,89.000000,331.250000
max,721.000000,18.000000,18.000000,255.000000,180.000000,188.000000,250.000000,531.000000


In [112]:
mapper = km.KeplerMapper(verbose=1)
# La proyeccion sobre el math score, corresponde a proyectar sobre la primera columna de data
projected_data = mapper.fit_transform(df2, projection=[1])  # Math score

KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: [1]
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (800, 8)

..Projecting data using: [1]


InvalidIndexError: (slice(None, None, None), array([1]))

In [ ]:
covering = km.Cover(n_cubes=7, perc_overlap=0.2)
covering

In [ ]:
G = mapper.map(
    projected_data, X=x, cover=covering, clusterer=sklearn.cluster.KMeans(n_clusters=2)
)

In [ ]:
G["meta_data"]

In [ ]:
mapper.visualize(
    G,
    title="Example 1 of Mapper on the Sphere dataset",
    color_values=x[:, 1],
    color_function_name="Height of the point",
    node_color_function=np.array(["average", "std", "sum", "max", "min"]),
)